In [ ]:
import pandas as pd
import psycopg2
import numpy as np

from psycopg2.extensions import register_adapter, AsIs
import psycopg2.extras

# Function to adapt numpy.int64 to PostgreSQL int
def adapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

# Register the adapter
register_adapter(np.int64, adapt_numpy_int64)


conn = psycopg2.connect(
    database="defaultdb",
    host="ibd-tp1-jssantosqueiroz-badd.f.aivencloud.com",
    user="avnadmin",
    password="",
    port="27074")

print('Connected to database')

try:
    df = pd.read_excel(
        './raw_data/RELATORIO_DTB_BRASIL_MUNICIPIO.xls', skiprows=6)

    print("reading excel file")

    # prints column names
    # print(df.columns)

    base_df = df.loc[:, ['UF', 'Nome_UF',
                        'Código Município Completo', 'Município', 'Nome_Município']]
    base_df
    query = """
    DROP TABLE IF EXISTS municipios;
    CREATE TABLE municipios (
        fk_cod_uf int ,
        codigo varchar(5),
        nome_municipio VARCHAR(255),
        codigo_completo char(7) PRIMARY KEY UNIQUE NOT NULL,
        CONSTRAINT fk_cod_uf FOREIGN KEY (fk_cod_uf) REFERENCES estados(cod_uf)

    );
    """
    print("creating table")
    cur = conn.cursor()
    cur.execute(query)
    print("table created")
    df = df[['UF', 'Município' , 'Nome_Município','Código Município Completo']]
    df.columns = ['fk_cod_uf', 'codigo', 'nome_municipio', 'codigo_completo']
    print(df.head())
    psycopg2.extras.execute_values(
        cur,
        """
        INSERT INTO municipios
            (fk_cod_uf, codigo, nome_municipio, codigo_completo) VALUES %s
        """, df.itertuples(index=False, name=None)
    )

    conn.commit()
    conn.close()
except Exception as e:
    print(e)
    print(e.query)
    print('Error on query')
    conn.close()

Connected to database
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
reading excel file
creating table
table created
   fk_cod_uf  codigo           nome_municipio  codigo_completo
0         11      15    Alta Floresta D'Oeste          1100015
1         11     379  Alto Alegre dos Parecis          1100379
2         11     403             Alto Paraíso          1100403
3         11     346         Alvorada D'Oeste          1100346
4         11      23                Ariquemes          1100023
